In [1]:
%pip install dotenv pandas langchain openai chromadb torch langchain_huggingface

Note: you may need to restart the kernel to use updated packages.


## RAG로 사용할 데이터 확인

### 1. 도서 정보 데이터

데이터는 공공API에서 제공하는 [국립중앙도서관 사서추천도서목록](https://www.nl.go.kr/NL/contents/N31101030900.do)을 사용한다.

xml 데이터를 json 데이터로 변환하였다.

In [2]:
import json

# JSON 데이터 로드
with open('./data/library_books.json', 'r', encoding='utf-8') as f:
    books = json.load(f)

print("\n" + "="*50)
print("📚 데이터 구조")
print("="*50)
print("\n📍 데이터 키 목록:")
print(", ".join(list(books[0].keys())))

print("\n📍 첫 번째 도서 상세 정보:")
for key, value in books[0].items():
    print(f"\n{key}:")
    # 긴 텍스트는 줄바꿈하여 보기 좋게 출력
    if len(str(value)) > 100:
        # 100자마다 줄바꿈
        formatted_value = '\n'.join([str(value)[i:i+100] for i in range(0, len(str(value)), 100)])
        print(f"    {formatted_value}")
    else:
        print(f"    {value}")

print("\n" + "="*50)
print(f"📚 전체 도서 수: {len(books)}권")
print("="*50 + "\n")


📚 데이터 구조

📍 데이터 키 목록:
category_code, category_name, title, author, publisher, isbn, contents, table_of_contents, publish_year, recommend_year, recommend_month

📍 첫 번째 도서 상세 정보:

category_code:
    6

category_name:
    인문과학

title:
    제대로 연습하는 법 : 어학부터 스포츠까지, 인지심리학이 제시하는 배움의 기술

author:
    아투로 E. 허낸데즈 지음 ;방진이 옮김

publisher:
    북트리거 지학사

isbn:
    9791193378335

contents:
    한때 유행했던 일만 시간의 법칙을 기억하는가? 어떤 분야에서 전문가가 되려면 최소한 일만 시간의 훈련이 필요하다는 개념이다. 하지만 단순히 연습의 양이 많다고 해서 모두가 전문가가
 되는 것은 아니다. 이 책은 심리학자이자 다중언어 구사자, 테니스 선수로도 활약했던 저자가 학습과 훈련, 그리고 기량 향상의 상관관계를 연구한 결과를 담고 있다. 장 피아제, 노
엄 촘스키, 그리고 일만 시간의 법칙을 제창한 심리학자 안데르스 에릭손 등 대가들의 이론 및 최신 연구 결과를 바탕으로, 뇌과학과 인지심리학적 관점에서 '제대로 연습하는 법'을 탐
구한다. 아마추어 스포츠 선수, 유명 체스 선수, 다중언어 구사자, 피아노 연주자 등 다양한 사례 분석을 통해 연습의 물리적 양보다 중요한 것은 질적인 측면임을 강조한다. 적절한 
휴식 속에서 배운 것을 재조합하고, 몰입 상태에서 연습할 때 비로소 '최고'라는 목표에 도달할 수 있는 것이다. 벌써 2025년이 100일 넘게 흘렀다. 완연한 봄을 맞이하여 새로
운 기술을 배우고 싶거나, 그동안 노력에 비해 실력이 늘지 않는다고 느껴왔다면, 이 책을 길잡이 삼아 ‘제대로 연습하는 법’을 배워보면 어떨까?

table_of_contents:
    서론 : 작은 

category 정보를 확인한다.

In [3]:
from collections import defaultdict
import pandas as pd
from IPython.display import display

# 카테고리 코드와 이름을 매핑하여 저장
category_map = defaultdict(int)
for book in books:
    category_key = (book['category_code'], book['category_name'])
    category_map[category_key] += 1

# 데이터프레임으로 변환
df_categories = pd.DataFrame([
    {
        '분류 코드': code,
        '분류명': name,
        '도서 수': count
    }
    for (code, name), count in category_map.items()
])

# 도서 수를 기준으로 내림차순 정렬
df_categories = df_categories.sort_values('도서 수', ascending=False)

# 스타일 적용
styled_df = df_categories.style\
    .set_properties(**{'text-align': 'center'})\
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center'), ('background-color', '#f0f0f0')]},
        {'selector': 'td', 'props': [('text-align', 'center')]},
    ])\
    .format({'도서 수': '{:,d}'})  # 천 단위 구분자 추가

# 테이블 출력
print(f"\n📚 총 분류 수: {len(df_categories)}개\n")
display(styled_df)


📚 총 분류 수: 6개



,분류 코드,분류명,도서 수
0,6,인문과학,342
1,5,사회과학,338
3,4,자연과학,319
2,11,어문학,296
4,425,,92
5,8,,1


chuck size를 설정하기 위해 텍스트 길이를 파악했다.

임베딩할 데이터는 category_name, title, contents, table_of_contents이다.

In [4]:
import pandas as pd

# 각 도서별 텍스트 길이 계산
text_lengths = []
for book in books:
    text = f"카테고리: {book['category_name']}\n제목: {book['title']}\n내용: {book['contents']}\n목차: {book['table_of_contents']}"
    length_info = {
        '카테고리': len(book['category_name']),
        '제목': len(book['title']),
        '내용': len(book['contents']),
        '목차': len(book['table_of_contents']),
        '전체': len(text)
    }
    text_lengths.append(length_info)

# DataFrame으로 변환
df_lengths = pd.DataFrame(text_lengths)

# 기본 통계 계산
stats = df_lengths.agg(['min', 'max', 'mean', 'median', 'std']).round(2)

# 결과 출력
print("\n=== 각 필드별 텍스트 길이 통계 ===")
display(stats)

# 백분위수 계산
percentiles = df_lengths['전체'].quantile([0.25, 0.5, 0.75, 0.9])
print("\n=== 전체 텍스트 길이 백분위수 ===")
for p, v in percentiles.items():
    print(f"{int(p*100)}번째 백분위: {int(v)}자")


=== 각 필드별 텍스트 길이 통계 ===


,카테고리,제목,내용,목차,전체
min,0.00,1.00,135.00,0.00,323.00
max,4.00,75.00,861.00,6677.00,7296.00
mean,3.52,14.16,495.10,884.01,1417.79
median,4.00,11.00,490.00,734.50,1264.50
std,1.03,9.65,95.41,740.71,746.49



=== 전체 텍스트 길이 백분위수 ===
25번째 백분위: 881자
50번째 백분위: 1264자
75번째 백분위: 1758자
90번째 백분위: 2343자


### 2. 사용자 정보

사용자 정보는 사용자가 읽은 책 정보와 사용자의 책 취향 정보를 사용한다.

#### 1. 사용자가 읽은 책 정보

In [5]:
with open('./data/test-case/user_reading_history.json', 'r', encoding='utf-8') as f:
    reading_history = json.load(f)

for i, book in enumerate(reading_history['read_books'], 1):
    print(f"\n📚 도서 {i}:")
    for key, value in book.items():
        print(f"  {key}: {value}")


📚 도서 1:
  title: 완벽이라는 중독 : 불안한 완벽주의자를 위한 심리학
  author: 토머스 커런 지음 ;김문주 옮김
  rating: 4.5
  review: 심리학적 관점에서 자아를 이해하는데 도움이 됨
  read_date: 2024-01-05
  genre: ['심리학', '자기계발']

📚 도서 2:
  title: 침묵을 배우는 시간 : 말이 넘쳐나는 세상 속, 더욱 빛을 발하는 침묵의 품격
  author: 코르넬리아 토프 지음 ;장혜경 옮김
  rating: 4.0
  review: 따뜻한 위로가 되는 에세이
  read_date: 2024-01-20
  genre: ['에세이', '자기계발']

📚 도서 3:
  title: 나답게 산다는 것 : 나를 찾고자 하는 이들의 철학수업
  author: 박은미 지음
  rating: 4.5
  review: 쉽게 읽을 수 있는 철학 에세이
  read_date: 2024-02-10
  genre: ['철학', '에세이']

📚 도서 4:
  title: 출근길 심리학 : 단단하고 유연한 멘탈을 위한 33가지 마음의 법칙
  author: 반유화 지음
  rating: 3.5
  review: 일상적인 심리학 이야기
  read_date: 2024-02-25
  genre: ['심리학', '자기계발']

📚 도서 5:
  title: 내가 누구인지 아는 것이 왜 중요한가
  author: 페터 베르 지음 ;장혜경 옮김
  rating: 4.0
  review: 자아 성찰에 도움이 되는 책
  read_date: 2024-03-08
  genre: ['심리학', '에세이']

📚 도서 6:
  title: 자화상 내 마음을 그리다
  author: 김선현 지음
  rating: 3.5
  review: 마음을 따뜻하게 해주는 이야기
  read_date: 2024-03-22
  genre: ['에세이']

📚 도서 7:
  title: 각본 없음 : 삶의 다음 페이지로 넘어가기 위해 쓴 

#### 2. 사용자의 책 취향 정보

In [6]:
with open('./data/test-case/user_reading_preferences.json', 'r', encoding='utf-8') as f:
    reading_preferences = json.load(f)

preferences = reading_preferences['preferences']

print("\n📚 선호 장르:")
for genre in preferences['genres']:
    print(f"  - {genre['name']} (가중치: {genre['weight']})")

print("\n📖 독서 스타일:")
for key, value in preferences['reading_style'].items():
    print(f"  - {key}: {value}")

print("\n🔑 관심 키워드:")
print(f"  {', '.join(preferences['keywords'])}")

print("\n⛔ 기피 주제:")
print(f"  {', '.join(preferences['avoid_topics'])}")


📚 선호 장르:
  - 심리학 (가중치: 0.8)
  - 자기계발 (가중치: 0.7)
  - 에세이 (가중치: 0.6)

📖 독서 스타일:
  - preferred_length: 중간
  - complexity_level: 쉬움
  - tone: 따뜻함

🔑 관심 키워드:
  불안, 힐링, 위로, 심리, 일상

⛔ 기피 주제:
  우울, 공포, 긴장감


### 벡터 DB 생성

In [7]:
import torch

# GPU 사용 가능 여부 확인 및 디바이스 설정
if torch.cuda.is_available():
    device = 'cuda'
    device_map = {'': 0}
elif torch.backends.mps.is_available():
    device = 'mps'
    device_map = {'': device}
else:
    device = 'cpu'
    device_map = {'': device}

print(f"사용 디바이스: {device}")

사용 디바이스: mps


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import Chroma
import json

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': device}
)

#--------------------- 1. 도서 정보 벡터 DB ---------------------
with open('./data/library_books.json', 'r', encoding='utf-8') as f:
    books = json.load(f)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        # 중앙값과 75퍼센타일 사이의 값으로 설정
    chunk_overlap=300,      # chunk_size의 20% 정도로 설정
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

book_documents = []
for book in books:
    text = f"카테고리: {book['category_name']}\n제목: {book['title']}\n내용: {book['contents']}\n목차: {book['table_of_contents']}"
    
    metadata = {
        'title': book['title'],
        'category': book['category_name'],
        'author': book['author'],
        'isbn': book['isbn'],
        'publish_year': book['publish_year'],
    }
    
    chunks = text_splitter.create_documents(
        texts=[text],
        metadatas=[metadata]
    )
    book_documents.extend(chunks)

# 도서 정보 벡터 DB 생성
books_vectorstore = Chroma.from_documents(
    documents=book_documents,
    embedding=embeddings,
    collection_name="books_collection"
)

print(f"도서 정보 벡터 DB 생성 완료: {len(book_documents)}개 문서")

#--------------------- 2. 독서 이력 벡터 DB ---------------------
with open('./data/test-case/user_reading_history.json', 'r', encoding='utf-8') as f:
    reading_history = json.load(f)

history_documents = []
for book in reading_history['read_books']:
    text = f"제목: {book['title']}\n저자: {book['author']}\n평점: {book['rating']}\n리뷰: {book['review']}\n장르: {', '.join(book['genre'])}\n읽은 날짜: {book['read_date']}"
    
    metadata = {
        'title': book['title'],
        'author': book['author'],
        'rating': book['rating'],
        'genre_str': ', '.join(book['genre']),
        'read_date': book['read_date']
    }
    
    doc = Document(page_content=text, metadata=metadata)
    history_documents.append(doc)

# 독서 이력 벡터 DB 생성
history_vectorstore = Chroma.from_documents(
    documents=history_documents,
    embedding=embeddings,
    collection_name="reading_history_collection"
)

print(f"독서 이력 벡터 DB 생성 완료: {len(history_documents)}개 문서")

#--------------------- 3. 독서 취향 벡터 DB ---------------------
with open('./data/test-case/user_reading_preferences.json', 'r', encoding='utf-8') as f:
    reading_preferences = json.load(f)

preferences_text = f"선호 장르: {', '.join([f'{g['name']}({g['weight']})' for g in reading_preferences['preferences']['genres']])}\n"
preferences_text += f"독서 스타일: 길이({reading_preferences['preferences']['reading_style']['preferred_length']}), "
preferences_text += f"복잡도({reading_preferences['preferences']['reading_style']['complexity_level']}), "
preferences_text += f"톤({reading_preferences['preferences']['reading_style']['tone']})\n"
preferences_text += f"관심 키워드: {', '.join(reading_preferences['preferences']['keywords'])}\n"
preferences_text += f"기피 주제: {', '.join(reading_preferences['preferences']['avoid_topics'])}"

preferences_metadata = {
    'genres_str': ', '.join([g['name'] for g in reading_preferences['preferences']['genres']]),
    'keywords_str': ', '.join(reading_preferences['preferences']['keywords']),
    'avoid_topics_str': ', '.join(reading_preferences['preferences']['avoid_topics'])
}

preferences_doc = Document(page_content=preferences_text, metadata=preferences_metadata)

# 독서 취향 벡터 DB 생성 (단일 문서)
preferences_vectorstore = Chroma.from_documents(
    documents=[preferences_doc],
    embedding=embeddings,
    collection_name="reading_preferences_collection"
)

print("독서 취향 벡터 DB 생성 완료: 1개 문서")


도서 정보 벡터 DB 생성 완료: 2201개 문서
독서 이력 벡터 DB 생성 완료: 15개 문서
독서 취향 벡터 DB 생성 완료: 1개 문서


### 추천 모델 구현

공통 프롬프트를 작성한다.

In [9]:
from langchain.prompts import ChatPromptTemplate

# 공통 기본 프롬프트 템플릿
BASE_RECOMMENDATION_TEMPLATE = """당신은 사용자의 독서 취향과 감정 상태에 맞춰 도서를 추천하는 전문가입니다.
다음 정보를 분석하여 사용자에게 가장 적합한 {num_recommendations}권의 책을 추천해주세요.

## 사용자 정보
- 현재 감정 상태: {user_emotion}
- 원하는 감정적 효과: {desired_emotional_effect}
- 직업: {occupation}
- 독서 상황: {reading_context}
- 선호하는 집중도: {focus_level}

{preferences_section}

{history_section}

## 후보 도서 목록
{candidate_books}

다음과 같은 척도로 각 후보 도서를 평가하세요:
1. 사용자의 현재 감정 상태와 원하는 감정적 효과와의 적합성
2. 사용자의 취향과의 일치도 (장르, 스타일, 선호 키워드)
3. 사용자의 기피 주제와 겹치지 않는지 여부
4. 사용자의 독서 이력과의 연관성
5. 독서 상황과 선호하는 집중도에 맞는지 여부
6. 사용자의 직업과 관련된 통찰이나 도움이 될 수 있는지 여부

최종 추천은 다음 형식으로 제시해주세요:
1. [첫 번째 추천 도서 제목] - 저자
   - 추천 이유: (사용자의 현재 감정과 원하는 효과를 고려한 구체적인 이유)
   - 이 책이 도움이 될 수 있는 이유: (감정적 효과나 얻을 수 있는 통찰 등)

2. [두 번째 추천 도서 제목] - 저자
   - 추천 이유: (사용자의 현재 감정과 원하는 효과를 고려한 구체적인 이유)
   - 이 책이 도움이 될 수 있는 이유: (감정적 효과나 얻을 수 있는 통찰 등)

3. [세 번째 추천 도서 제목] - 저자
   - 추천 이유: (사용자의 현재 감정과 원하는 효과를 고려한 구체적인 이유)
   - 이 책이 도움이 될 수 있는 이유: (감정적 효과나 얻을 수 있는 통찰 등)
"""

# 공통 프롬프트 템플릿 생성
base_prompt_template = ChatPromptTemplate.from_template(BASE_RECOMMENDATION_TEMPLATE)


사용자 독서 이력과 후보 도서 정보를 프롬프트에 추가하기 위해 format 함수를 추가한다.

In [50]:
def format_reading_history(reading_history_data):
    if not reading_history_data:
        return "관련 독서 이력이 없습니다."
    
    formatted_entries = []
    for item in reading_history_data:
        # 다단계 RAG 결과 처리
        if isinstance(item, dict) and 'metadata' in item:
            entry = {
                'title': item['metadata'].get('title', '제목 없음'),
                'author': item['metadata'].get('author', '저자 미상'),
                'genre': item['metadata'].get('genre_str', '장르 없음'),
                'rating': item['metadata'].get('rating', 0),
                'content': item['content'][:200] + "..." if len(item['content']) > 200 else item['content']
            }
        # 직접 벡터 검색 결과 처리
        elif isinstance(item, dict) and 'title' in item:
            entry = {
                'title': item['title'],
                'author': item['author'],
                'genre': item['genre'],
                'rating': item['rating'],
                'content': item['review']
            }
        else:
            continue
            
        formatted_entries.append(
            f"- '{entry['title']}' (저자: {entry['author']}, 장르: {entry['genre']}, 평점: {entry['rating']})\n"
            f"  내용: {entry['content']}"
        )
    
    return "\n\n".join(formatted_entries) if formatted_entries else "관련 독서 이력이 없습니다."

def format_book_candidates(candidates_data):
    if not candidates_data:
        return "추천할 만한 후보 도서가 없습니다."
    
    formatted_entries = []
    for i, item in enumerate(candidates_data, 1):
        # Document 객체와 score 튜플 처리
        if isinstance(item, tuple) and hasattr(item[0], 'metadata'):
            doc, score = item
            entry = {
                'title': doc.metadata.get('title', '제목 없음'),
                'author': doc.metadata.get('author', '저자 미상'),
                'category': doc.metadata.get('category', '카테고리 없음'),
                'content': doc.page_content,
                'score': score
            }
        # 직접 벡터 검색 결과 처리 (딕셔너리)
        elif isinstance(item, dict) and 'metadata' in item:
            entry = {
                'title': item['metadata'].get('title', '제목 없음'),
                'author': item['metadata'].get('author', '저자 미상'),
                'category': item['metadata'].get('category', '카테고리 없음'),
                'content': item['content'],
                'score': item.get('score', 0.0)
            }
        else:
            continue
            
        formatted_entries.append(
            f"{i}. '{entry['title']}' (저자: {entry['author']}, 카테고리: {entry['category']}, 유사도: {entry['score']:.2f})\n"
            f"   내용: {entry['content']}"
        )
    
    return "\n\n".join(formatted_entries) if formatted_entries else "추천할 만한 후보 도서가 없습니다."

llm 모델을 정의한다.

In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    top_p=0.9,
    max_tokens=512,
    api_key=api_key
)

chain = base_prompt_template | llm

모델의 실행 결과를 저장하기 위한 함수를 선언한다.

In [13]:
from datetime import datetime
import pandas as pd

def get_existing_recommendation(model, method, num_candidates):
    filename = f"recommendations_{model}_{method}_num-candidates_{num_candidates}.csv"
    
    if os.path.exists(filename):
        print(f"\n동일한 모델({model})과 방법({method})의 결과가 이미 존재합니다.")
        existing_df = pd.read_csv(filename)
        return existing_df['result'].iloc[0]
    
    return None

def save_recommendation_log(log_data):
    filename = f"recommendations_{log_data['model']}_{log_data['method']}_num-candidates_{log_data['num_candidates']}.csv"
    new_df = pd.DataFrame([log_data])
    new_df.to_csv(filename, index=False)
    print(f"\n추천 결과가 {filename}에 추가되었습니다.")

#### 1. RAG에 기반한 추천 로직

In [20]:
def multi_stage_rag_search(user_query, num_candidates, num_history_results):
    results = {}
    
    # 1단계: 항상 사용자 취향 정보 검색
    preference_results = preferences_vectorstore.similarity_search_with_score(user_query, k=1)
    if preference_results:
        results['user_preferences'] = preference_results[0]
    
    # 2단계: 사용자의 독서 이력에서 관련 도서 검색
    history_results = history_vectorstore.similarity_search_with_score(user_query, k=num_history_results)
    if history_results:
        results['reading_history'] = history_results
    
    # 3단계: 독서 이력과 취향 정보를 고려하여 강화된 쿼리 생성
    enhanced_query = user_query
    
    # 취향 정보에서 관련 키워드 추출
    if 'user_preferences' in results:
        pref_doc = results['user_preferences'][0]
        keywords = pref_doc.metadata.get('keywords_str', '')
        if keywords:
            enhanced_query += f" 키워드: {keywords}"
    
    # 독서 이력에서 높은 평점의 장르 추출
    if 'reading_history' in results:
        liked_genres = []
        for doc, _ in results['reading_history']:
            if doc.metadata.get('rating', 0) >= 4.0:
                genres = doc.metadata.get('genre_str', '').split(', ')
                liked_genres.extend(genres)
        
        if liked_genres:
            unique_genres = list(set(liked_genres))
            enhanced_query += f" 선호 장르: {', '.join(unique_genres[:3])}"
    
    print(f"강화된 쿼리: {enhanced_query}")
    
    # 4단계: 강화된 쿼리로 도서 검색
    book_results = books_vectorstore.similarity_search_with_score(enhanced_query, k=num_candidates)
    
    # 결과 포맷팅 - 추천 도서만 포함
    formatted_results = []
    for book_doc, book_score in book_results:
        formatted_results.append({
            'content': book_doc.page_content,
            'metadata': book_doc.metadata,
            'score': book_score
        })
    
    return formatted_results

In [55]:
from langchain.callbacks import get_openai_callback

def multi_stage_recommendation(user_emotion, desired_emotional_effect, occupation, reading_context, focus_level, num_candidates=8, num_history_results=3, num_recommendations=3):
    existing_result = get_existing_recommendation(llm.model_name, 'multi_stage', num_candidates)
    if existing_result is not None:
        return existing_result
    
    # 감정과 원하는 효과를 기반으로 검색 쿼리 구성
    search_query = f"{user_emotion}을 느끼는 사람이 {desired_emotional_effect}할 수 있는 책"
    
    # 다단계 RAG 검색 수행
    book_candidates = multi_stage_rag_search(search_query, num_candidates, num_history_results)
    
    candidates_formatted = format_book_candidates(book_candidates)
    
    # 프롬프트에 전달할 입력값
    prompt_inputs = {
        "user_emotion": user_emotion,
        "desired_emotional_effect": desired_emotional_effect,
        "occupation": occupation,
        "reading_context": reading_context,
        "focus_level": focus_level,
        "preferences_section": '',
        "history_section": '',
        "candidate_books": candidates_formatted,
        "num_recommendations": num_recommendations
    }
    
    # 실제 프롬프트 생성
    prompt_text = base_prompt_template.format(**prompt_inputs)
    
    # OpenAI 콜백을 사용하여 토큰 사용량 추적
    with get_openai_callback() as cb:
        result = chain.invoke(prompt_inputs)
        
        # 토큰 사용량 정보 가져오기
        token_usage = {
            'prompt_tokens': cb.prompt_tokens,
            'completion_tokens': cb.completion_tokens,
            'total_tokens': cb.total_tokens,
            'total_cost': cb.total_cost
        }

    # 결과 로깅
    log_data = {
        'timestamp': datetime.now(),
        'model': llm.model_name,
        'method': 'multi_stage',
        'num_candidates': num_candidates,
        # 프롬프트 정보
        'prompt': prompt_text,
        'prompt_inputs': {
            "user_emotion": user_emotion,
            "desired_emotional_effect": desired_emotional_effect,
            "occupation": occupation,
            "reading_context": reading_context,
            "focus_level": focus_level
        },
        # 사용자 입력 정보 추가
        'user_emotion': user_emotion,
        'desired_emotional_effect': desired_emotional_effect,
        'occupation': occupation,
        'reading_context': reading_context,
        'focus_level': focus_level,
        # 추천 관련 정보
        'book_candidates': book_candidates,
        'result': result.content,
        # 토큰 사용량
        'token_usage': token_usage
    }
    
    # 새로운 결과 저장
    save_recommendation_log(log_data)
    
    return result.content

#### 2. Prompt 기반의 추천 로직

In [22]:
# 취향 정보를 문자열로 포맷팅 (프롬프트에 직접 사용)
preferences_formatted = f"""
## 사용자 독서 취향 정보
- 선호 장르: {', '.join([f"{g['name']}(가중치:{g['weight']})" for g in reading_preferences['preferences']['genres']])}
- 선호 독서 스타일: 
  * 선호 길이: {reading_preferences['preferences']['reading_style']['preferred_length']}
  * 복잡도: {reading_preferences['preferences']['reading_style']['complexity_level']}
  * 선호 톤: {reading_preferences['preferences']['reading_style']['tone']}
- 관심 키워드: {', '.join(reading_preferences['preferences']['keywords'])}
- 기피 주제: {', '.join(reading_preferences['preferences']['avoid_topics'])}
"""

print("사용자 취향 정보 추출 완료")

사용자 취향 정보 추출 완료


In [56]:
def direct_vector_recommendation(user_emotion, desired_emotional_effect, occupation, reading_context, focus_level, num_candidates=8, num_history_results=3, num_recommendations=3):
    existing_result = get_existing_recommendation(llm.model_name, 'direct', num_candidates)
    if existing_result is not None:
        return existing_result
    
    # 감정과 원하는 효과를 기반으로 검색 쿼리 구성
    search_query = f"{user_emotion}을 느끼는 사람이 {desired_emotional_effect}할 수 있는 책"
    
    # 1. 쿼리에 관련된 도서 검색
    book_results = books_vectorstore.similarity_search_with_score(search_query, k=num_candidates)
    
    # 2. 사용자의 독서 이력 중 관련 있는 책 검색
    history_results = history_vectorstore.similarity_search_with_score(search_query, k=num_history_results)
    
    # 3. 검색 결과 포맷팅
    candidates_formatted = format_book_candidates(book_results)

    history_section = f"## 사용자가 읽은 관련 책들\n{format_reading_history(history_results)}"
    
    # 프롬프트에 전달할 입력값
    prompt_inputs = {
        "user_emotion": user_emotion,
        "desired_emotional_effect": desired_emotional_effect,
        "occupation": occupation,
        "reading_context": reading_context,
        "focus_level": focus_level,
        "preferences_section": preferences_formatted,
        "history_section": history_section,
        "candidate_books": candidates_formatted,
        "num_recommendations": num_recommendations
    }
    
    # 실제 프롬프트 생성
    prompt_text = base_prompt_template.format(**prompt_inputs)
    
    # OpenAI 콜백을 사용하여 토큰 사용량 추적
    with get_openai_callback() as cb:
        result = chain.invoke(prompt_inputs)
        
        # 토큰 사용량 정보 가져오기
        token_usage = {
            'prompt_tokens': cb.prompt_tokens,
            'completion_tokens': cb.completion_tokens,
            'total_tokens': cb.total_tokens,
            'total_cost': cb.total_cost
        }

    # 결과 로깅
    log_data = {
        'timestamp': datetime.now(),
        'model': llm.model_name,
        'method': 'direct',
        'num_candidates': num_candidates,
        # 프롬프트 정보
        'prompt': prompt_text,
        'prompt_inputs': {
            "user_emotion": user_emotion,
            "desired_emotional_effect": desired_emotional_effect,
            "occupation": occupation,
            "reading_context": reading_context,
            "focus_level": focus_level
        },
        # 사용자 입력 정보 추가
        'user_emotion': user_emotion,
        'desired_emotional_effect': desired_emotional_effect,
        'occupation': occupation,
        'reading_context': reading_context,
        'focus_level': focus_level,
        # 추천 관련 정보
        'book_candidates': book_results,
        'result': result.content,
        # 토큰 사용량
        'token_usage': token_usage
    }
    
    # 새로운 결과 저장
    save_recommendation_log(log_data)
    
    return result.content

In [44]:
def get_book_recommendations(user_emotion, desired_emotional_effect, occupation, reading_context, focus_level, method="both"):
    print(f"현재 감정 상태: {user_emotion}")
    print(f"원하는 감정적 효과: {desired_emotional_effect}")
    print(f"직업: {occupation}")
    print(f"독서 상황: {reading_context}")
    print(f"선호하는 집중도: {focus_level}")
    
    results = {}

    num_candidates = 6
    num_history_results = 3
    num_recommendations = 3

    if method in ["multi_stage", "both"]:
        print("\n다단계 RAG 기반 추천 도서를 찾는 중...\n")
        multi_stage_result = multi_stage_recommendation(
            user_emotion=user_emotion,
            desired_emotional_effect=desired_emotional_effect,
            occupation=occupation,
            reading_context=reading_context,
            focus_level=focus_level,
            num_candidates=num_candidates,
            num_history_results=num_history_results,
            num_recommendations=num_recommendations
        )
        results["multi_stage"] = multi_stage_result
    
    if method in ["direct", "both"]:
        print("\n직접 벡터 검색 기반 추천 도서를 찾는 중...\n")
        direct_result = direct_vector_recommendation(
            user_emotion=user_emotion,
            desired_emotional_effect=desired_emotional_effect,
            occupation=occupation,
            reading_context=reading_context,
            focus_level=focus_level,
            num_candidates=num_candidates,
            num_history_results=num_history_results,
            num_recommendations=num_recommendations
        )
        results["direct"] = direct_result
    
    return results

In [54]:
# 테스트 시나리오 (README.md의 TestCase1에 맞게 수정)
test_results = get_book_recommendations(
    user_emotion="불안함",                      # 질문 1 응답
    desired_emotional_effect="위로와 안정감",     # 질문 2 응답
    occupation="직장인",                       # 질문 3 응답
    reading_context="잠들기 전 독서",            # 질문 4 응답
    focus_level="가볍게 읽을 수 있는 책",         # 질문 5 응답
    method="both"  # 두 방식 모두 사용
)

print("\n=== 다단계 RAG 기반 도서 추천 결과 ===\n")
print(test_results.get("multi_stage", "결과 없음"))

print("\n=== 직접 벡터 검색 기반 도서 추천 결과 ===\n")
print(test_results.get("direct", "결과 없음"))

현재 감정 상태: 불안함
원하는 감정적 효과: 위로와 안정감
직업: 직장인
독서 상황: 잠들기 전 독서
선호하는 집중도: 가볍게 읽을 수 있는 책

다단계 RAG 기반 추천 도서를 찾는 중...

강화된 쿼리: 불안함을 느끼는 사람이 위로와 안정감할 수 있는 책 키워드: 불안, 힐링, 위로, 심리, 일상 선호 장르: 역사, 여행, 문화
----book_results----
[{'content': '카테고리: 인문과학\n제목: 잠깐 머리 좀 식히고 오겠습니다\n내용: 힐링, 자존감, 긍정, 신경 끄기 등의 말이 꾸준히 유행하는 이유는 아직도 많은 사람들이 일상에서 얻는 스트레스와 무기력을 어떻게 극복해야 할지 모르고 있기 때문일 것이다. 정신과 전문의인 이 책의 저자는 스트레스 관리란 마음 관리라 말하며, 그동안 상담해 왔던 수많은 사람들의 고민 중 대표적인 것들을 모아 질의응답 형식의 심리 처방전을 제공했다. 대부분의 사람들은 직장생활, 인간관계, 자신이나 타인의 습관과 태도, 조절되지 않는 감정, 낮은 자존감 때문에 행복하지 않음을 느낀다. 저자는 우리 마음이 스트레스 받는 이유가 너무 열심히 살고 있기 때문이며, 그런 마음을 잘 이해해 주고 마음이 즐거운 일을 해 줘야 한다고 말한다. 이 책의 사연들은 ‘나와 같은 사람이 많구나’ 라는 공감과 위안을 줄 것이다. 또 유쾌하지만 구체적이고 진솔한 저자의 처방은 일상의 스트레스를 이겨 낼 수 있는 긍정적인 마음을 심어 줄 것이다.', 'metadata': {'author': '윤대현', 'category': '인문과학', 'isbn': '9788965746584', 'publish_year': '2018', 'title': '잠깐 머리 좀 식히고 오겠습니다'}, 'score': 4.433990001678467}, {'content': "카테고리: \n제목: 나에게 오늘을 선물합니다\n내용: 인생에는 위로가 필요한 순간들이 있다. 인생이 어렵고 삶이 불안할 때, 어떻게 살아야 할지 고민일 때, 나만 늘 제자리라는

### 평가

평가를 위해 결과를 저장한 csv 파일에서 도서 정보를 추출한다.

In [70]:
import pandas as pd
import json
import re
import requests
from dotenv import load_dotenv
import os

def extract_books_from_result(result):
    # 두 가지 형식 모두 찾을 수 있는 패턴:
    # 1) **[제목] - 저자** 형식
    # 2) **'제목'** - 저자 형식
    pattern = r'\*\*(?:\[(.*?)\]|\'(.*?)\')\*\* - (.*?)(?=\n|$)'
    matches = re.findall(pattern, result)
    
    # 결과 처리
    books = []
    for match in matches:
        # match[0]은 대괄호 형식의 제목
        # match[1]은 따옴표 형식의 제목
        # match[2]는 저자
        title = match[0] if match[0] else match[1]
        author = match[2]
        books.append((title, author))
    
    return books

def find_isbn_from_vectordb(books, books_vectorstore):
    matched_books = []
    
    for title, author in books:
        # 저자 이름에서 '지음' 등의 부가 설명 제거
        author_clean = re.split(r'\s+지음', author)[0]
        
        # 제목과 저자로 검색 쿼리 구성
        search_query = f"제목: {title} 저자: {author_clean}"
        
        # 벡터 DB에서 가장 유사한 문서 검색
        results = books_vectorstore.similarity_search_with_score(search_query, k=1)
        
        print(results)
        if results:
            doc, score = results[0]
            # 유사도 점수가 일정 임계값 이상인 경우만 매칭으로 간주
            if score < 1.5:  # 임계값은 조정 가능
                matched_books.append({
                    'title': title,
                    'author': author,
                    'isbn': doc.metadata.get('isbn'),
                    'similarity_score': score
                })
    
    return matched_books


def normalize_text(text):
    return ''.join(text.split()).lower()

def find_books_info(recommended_books):
    matched_books = []

    with open('./data/library_books.json', 'r', encoding='utf-8') as f:
        library_data = json.load(f)
    
    for title, author in recommended_books:
        # 검색할 도서의 제목과 저자 정규화
        search_title = normalize_text(title)
        search_author = normalize_text(author)
        
        # 매칭되는 도서 찾기
        for book in library_data:
            lib_title = normalize_text(book['title'])
            lib_author = normalize_text(book['author'])
            
            # 제목이 포함되어 있고, 저자명이 포함되어 있으면 매칭으로 간주
            if (search_title in lib_title or lib_title in search_title) and \
               (search_author in lib_author or lib_author in search_author):
                matched_books.append({
                    'title': title,  # 원본 제목 유지
                    'author': author,  # 원본 저자명 유지
                    'isbn': book['isbn'],
                    'matched_library_title': book['title'],  # 매칭된 도서관 데이터의 제목
                    'matched_library_author': book['author']  # 매칭된 도서관 데이터의 저자
                })
                break
    
    return matched_books


In [71]:
def get_aladin_book_info(isbn):
    load_dotenv()

    ALADIN_API_KEY = os.getenv('ALADIN_API_KEY')
    
    if not ALADIN_API_KEY:
        raise ValueError("알라딘 API 키가 설정되지 않았습니다. .env 파일을 확인해주세요.")
    
    url = "http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx"
    params = {
        'ttbkey': ALADIN_API_KEY,
        'itemIdType': 'ISBN13',
        'ItemId': isbn,
        'Output': 'js',
        'Version': '20131101',
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API 호출 중 오류 발생: {e}")
        return None

In [72]:
def collect_book_info(csv_path):
    print(f"csv: {csv_path}")
    save_path = csv_path.rsplit('.', 1)[0] + '_book_info.json'
    
    # CSV 파일에서 result 읽기
    df = pd.read_csv(csv_path)
    result = df['result'].iloc[0]
    
    # 추천 도서 추출
    recommended_books = extract_books_from_result(result)
    print(f"추출된 도서: {recommended_books}")
    
    # ISBN 매칭 (벡터 DB 사용)
    books_with_isbn = find_books_info(recommended_books)
    print(f"\nISBN 매칭 결과: {books_with_isbn}")
    
    # 알라딘 API 호출 및 결과 저장
    aladin_results = []
    for book in books_with_isbn:
        info = get_aladin_book_info(book['isbn'])
        if info:
            aladin_results.append({
                'original_recommendation': book,
                'aladin_info': info
            })
    
    # 결과 저장
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(aladin_results, f, ensure_ascii=False, indent=2)
    
    print(f"\n알라딘 API 결과가 저장되었습니다. 총 {len(aladin_results)}개 도서 정보 수집")
    return aladin_results

In [73]:
collect_book_info("./recommendations_gpt-4o-mini_direct_num-candidates_6.csv")
collect_book_info("./recommendations_gpt-4o-mini_multi_stage_num-candidates_6.csv")

csv: ./recommendations_gpt-4o-mini_direct_num-candidates_6.csv
추출된 도서: []

ISBN 매칭 결과: []

알라딘 API 결과가 저장되었습니다. 총 0개 도서 정보 수집
csv: ./recommendations_gpt-4o-mini_multi_stage_num-candidates_6.csv
추출된 도서: [('잠깐 머리 좀 식히고 오겠습니다', '윤대현  '), ('나에게 오늘을 선물합니다', '김나위  '), ('마음은 괜찮냐고 시가 물었다 : 시 읽어주는 정신과 의사가 건네는 한 편의 위로', '황인환  ')]

ISBN 매칭 결과: [{'title': '잠깐 머리 좀 식히고 오겠습니다', 'author': '윤대현  ', 'isbn': '9788965746584', 'matched_library_title': '잠깐 머리 좀 식히고 오겠습니다', 'matched_library_author': '윤대현'}, {'title': '나에게 오늘을 선물합니다', 'author': '김나위  ', 'isbn': '9791190456425', 'matched_library_title': '나에게 오늘을 선물합니다', 'matched_library_author': '김나위'}, {'title': '마음은 괜찮냐고 시가 물었다 : 시 읽어주는 정신과 의사가 건네는 한 편의 위로', 'author': '황인환  ', 'isbn': '9791192097053', 'matched_library_title': '마음은 괜찮냐고 시가 물었다 : 시 읽어주는 정신과 의사가 건네는 한 편의 위로', 'matched_library_author': '황인환'}]

알라딘 API 결과가 저장되었습니다. 총 3개 도서 정보 수집


[{'original_recommendation': {'title': '잠깐 머리 좀 식히고 오겠습니다',
   'author': '윤대현  ',
   'isbn': '9788965746584',
   'matched_library_title': '잠깐 머리 좀 식히고 오겠습니다',
   'matched_library_author': '윤대현'},
  'aladin_info': {'version': '20131101',
   'logo': 'http://image.aladin.co.kr/img/header/2011/aladin_logo_new.gif',
   'title': '알라딘 상품정보 - 잠깐 머리 좀 식히고 오겠습니다',
   'link': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=151352195&amp;partner=openAPI',
   'pubDate': 'Mon, 19 May 2025 04:51:13 GMT',
   'totalResults': 1,
   'startIndex': 1,
   'itemsPerPage': 1,
   'query': 'isbn13=9788965746584',
   'searchCategoryId': 0,
   'searchCategoryName': '',
   'item': [{'title': '잠깐 머리 좀 식히고 오겠습니다',
     'link': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=151352195&amp;partner=openAPI&amp;start=api',
     'author': '윤대현 (지은이)',
     'pubDate': '2018-06-18',
     'description': '많은 사람들의 심리에 명쾌한 처방과 따뜻한 위로를 건네왔던 서울대병원 정신과 전문의 윤대현 교수는 진짜 중요한 것은 스트레스를 없애거나 피하는 것이 아니라 그것을 잘 받아들이는 것이라고 말한다.',
  

In [74]:
def create_evaluation_prompt(csv_path):
    df = pd.read_csv(csv_path)
    prompt = df['prompt'].iloc[0]
    result = df['result'].iloc[0]
    
    # 사용자 정보와 독서 취향 정보 추출
    user_info = ""
    reading_preferences = ""
    
    # prompt에서 섹션 추출
    sections = prompt.split("##")
    for section in sections:
        if "사용자 정보" in section:
            user_info = section.strip()
        elif "사용자 독서 취향 정보" in section:
            reading_preferences = section.strip()
    
    # 도서 정보 JSON 파일 읽기
    book_info_path = csv_path.rsplit('.', 1)[0] + '_book_info.json'
    with open(book_info_path, 'r', encoding='utf-8') as f:
        book_info = json.load(f)
    
    # 프롬프트 템플릿 생성
    evaluation_prompt = f"""도서 추천 시스템 평가

도서 추천 시스템의 성능을 평가해주세요.
질문에 대한 사용자의 답변에 기반해 도서를 추천하는 서비스를 개발할 예정입니다.

## {user_info}

## {reading_preferences}

## 추천된 도서 목록
{result}

## 추천된 도서 상세 정보 (알라딘 도서 정보)
{json.dumps(book_info, ensure_ascii=False, indent=2)}

## 평가 요청사항
1. 사용자의 답변과 사용자의 독서 취향 정보를 고려하여 추천된 책의 관련성을 0~5점 사이로 평가해주세요. 도서 상세 정보가 없는 경우 0점을 부여합니다.
2. 각 도서별로 다음 기준에 따라 평가해주세요:
- 사용자의 현재 감정 상태와의 적합성
- 사용자가 원하는 감정적 효과와의 연관성
- 사용자의 독서 취향과의 일치도
- 독서 상황(예: 잠들기 전 독서)과의 적합성
- 선호하는 집중도와의 부합성
3. 전반적인 추천 시스템의 성능에 대한 의견을 제시해주세요.
4. 개선이 필요한 부분이 있다면 구체적으로 제안해주세요.

평가 결과는 향후 추천 시스템 개선에 활용될 예정입니다.
"""
    
    # 프롬프트를 파일로 저장
    prompt_save_path = csv_path.rsplit('.', 1)[0] + '_evaluation_prompt.txt'
    with open(prompt_save_path, 'w', encoding='utf-8') as f:
        f.write(evaluation_prompt)
    
    print(f"평가 프롬프트가 {prompt_save_path}에 저장되었습니다.")
    return evaluation_prompt

In [75]:
create_evaluation_prompt('./recommendations_gpt-4o-mini_direct_num-candidates_6.csv')
create_evaluation_prompt("./recommendations_gpt-4o-mini_multi_stage_num-candidates_6.csv")

평가 프롬프트가 ./recommendations_gpt-4o-mini_direct_num-candidates_6_evaluation_prompt.txt에 저장되었습니다.
평가 프롬프트가 ./recommendations_gpt-4o-mini_multi_stage_num-candidates_6_evaluation_prompt.txt에 저장되었습니다.


'도서 추천 시스템 평가\n\n도서 추천 시스템의 성능을 평가해주세요.\n질문에 대한 사용자의 답변에 기반해 도서를 추천하는 서비스를 개발할 예정입니다.\n\n## 사용자 정보\n- 현재 감정 상태: 불안함\n- 원하는 감정적 효과: 위로와 안정감\n- 직업: 직장인\n- 독서 상황: 잠들기 전 독서\n- 선호하는 집중도: 가볍게 읽을 수 있는 책\n\n## \n\n## 추천된 도서 목록\n1. **\'잠깐 머리 좀 식히고 오겠습니다\'** - 윤대현  \n   - 추천 이유: 이 책은 스트레스와 불안감을 관리하는 방법에 대한 심리적 처방을 제공하며, 독자가 겪고 있는 불안함을 이해하고 위로해 줄 수 있습니다. 저자는 직장생활과 인간관계에서 오는 스트레스를 다루며, 공감과 위안을 통해 독자의 마음을 안정시켜 줄 것입니다.  \n   - 이 책이 도움이 될 수 있는 이유: 직장인으로서 일상에서 느끼는 스트레스에 대한 구체적인 사례와 긍정적인 마음을 심어줄 수 있는 조언이 담겨 있어, 독자가 스스로 위로받고 일상에서 겪는 불안을 덜어내는 데 큰 도움이 될 것입니다.  \n\n2. **\'나에게 오늘을 선물합니다\'** - 김나위  \n   - 추천 이유: 이 책은 힘든 순간에 자신을 위로하는 방법과 함께, 우리가 혼자가 아니라는 사실을 깨닫게 해줍니다. 불안한 감정 상태를 가진 독자에게 큰 위로와 안정감을 제공할 수 있는 내용이 담겨 있습니다.  \n   - 이 책이 도움이 될 수 있는 이유: 일상에서 겪는 어려움에 대한 다른 사람들의 이야기를 통해 공감하고 위로받을 수 있으며, 스스로를 돌아보는 시간을 가질 수 있어 감정적 안정감을 찾는 데 기여할 것입니다.  \n\n3. **\'마음은 괜찮냐고 시가 물었다 : 시 읽어주는 정신과 의사가 건네는 한 편의 위로\'** - 황인환  \n   - 추천 이유: 이 책은 정신건강 전문의가 쓴 시를 통해 독자의 마음을 위로하고, 복잡한 감정을 이해하도록 돕습니다. 독자가 느끼는 불안과 외로움을 공감하며, 그것들을 치유하는 